<a href="https://colab.research.google.com/github/Simu96/2020-2-OSSP1-WhatsUp-5/blob/sim/Data_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Crawling

In [2]:
# 환경 구축
!pip install selenium
!pip install BeautifulSoup4
!apt update
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 2.8MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic I

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException

from bs4 import BeautifulSoup
import time
import csv

In [4]:
#공공보건 포털사이트의 뉴스 기사를 크롤링함
site = 'http://health.chosun.com/list.html?menu=&more_menu=&more_smenu=&nowcode=1&type=&pn=2757'

#csv 파일 저장을 위한 f, utf-8을 인코딩으로 사용
f = open('health_chosun.csv', 'w',encoding='utf-8')
#파일 객체를 생성
wr = csv.writer(f)

In [5]:
row = 1

#chrome으로 driver를 실행
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

#조건 성립 시간을 위한 wait 객체 설정
wait = WebDriverWait(driver,20)

#driver의 브라우저 크기를 최대화
driver.maximize_window()
#최대화를 하지 않았을 경우 마우스 포인터가 정확히 div를 클릭하지 못하여 크롤링을 못할 수 있음

#웹의 컴포넌트 로딩을 기다리기 위한 implicitly_wait 함수, 로딩이 끝나면 바로 진행된다.
driver.implicitly_wait(3)

#미리 설정해둔 공공보건 포털사이트로 이동한다.
driver.get(site)

#기사의 제목, 내용, 생성날짜를 저장할 변수를 설정한다.
name=""
content=""
date=""

while True:
    time.sleep(1)
    list_num = len(driver.find_elements_by_css_selector("dl.list_item"))
    while True:
        #변수들의 초기화
        name=""
        content=""
        date=""
        while True:
            news_list = driver.find_element_by_css_selector("div#list_area dl:nth-child(" + str(list_num) +")")
            news = news_list.find_element_by_css_selector('dt a')
            news_text = news_list.text
            #잡다한 만화, 책광고 등등의 기사를 제외시킴
            if news.text.find("카툰") != -1 or news.text.find("해랑")!= -1 or news.text.find("서적") != -1 or news.text.find("튼튼선생") != -1 or news.text.find("양냥") != -1 or news_text.find("비만클리닉") != -1:
                print(str(row) + "오류발생")
                list_num -= 1
                if(list_num < 1):
                    break;
            else:
                break;      
        if(list_num < 1):
            break;
        news.click()
        try:
            alert = driver.switch_to.alert
            alert.accept()
            time.sleep(2)
        except :
            pass
        try :
            time.sleep(1)
            name = driver.find_element_by_css_selector("h1#news_title_text_id").text
            date = driver.find_element_by_css_selector("p#date_text").text.split(" ")[1]
            parts = driver.find_elements_by_css_selector("div.par p")
            for part in parts:
                if part.text.find("기자") == -1 and part.text.replace(" ","") != "":
                    content += part.text
                else:
                    break;
            wr.writerow([row, name, date, content])
            print(str(row) + " " + name)
            row +=1
            driver.back()
            driver.implicitly_wait(5)
            list_num -= 1
            if(list_num < 1):
                break;            
        except (AttributeError, NoSuchElementException, UnexpectedAlertPresentException):
            driver.back()
            list_num -= 1
            if(list_num < 1):
                break;
            
    page_now = driver.find_element_by_css_selector("div.paginate ul.paginate_num span.listAct").text.split(" ")[0]
    url = "http://health.chosun.com/list.html?menu=&more_menu=&more_smenu=&nowcode=1&type=&pn=" + str(int(page_now) -1)
    driver.get(url)

1 업그레이드 암치료/ 간암
2 ‘우리집이 바로 헬스클럽’ 실내운동기구 가이드
3 업그레이드 암치료/ 유방암
4 뉴스초첨/ 盧당선자가 받은 척추수술
5 겨울철 마라톤 주의사항
6 겨울이면 사과처럼 붉어지는 얼굴 대처법
7 자궁경부암/ 수술 받고도 아기 낳을 수 있다
8 [건강 허와실] 소금 양치질, 되레 치아 손상 일으켜
9 남성들에게도 갱년기가 있을까?
10 사랑의 질투인가 망상인가…의처증-의부증
11 임신부 비만-고혈압-당뇨…뱃속 아기가 위험
12 혈압 높아 두통-뒷목 뻣뻣한 것 아니다
13 업그레이드 암치료/ 폐암
14 “유방암 수술전 초음파 검사 받아라”
15 대상포진, 72시간내 바이러스 치료제 투여해야
16 당뇨병 환자, 운동할 때도 ‘조심 조심’
17 업그레이드 암치료…췌장암
18 ‘성대 마비증’ 수술로 정상적 목소리 되찾아
19 햇볕 많이 쬐면 피부노화
20 업그레이드 암치료/ 전립선암
21 업그레이드 암치료/ 소아 백혈병
22 울퉁불퉁 불거진 다리 핏줄 수술로 제거해야
23 편집실/ 어린이 당뇨환자 주위 보살핌 중요
24 어린이 당뇨 환자 급속 확산
25 뚱뚱한 남자가 통풍 잘 걸린다
26 “우리집 개구쟁이 ADHD증상 아닐까?”
27 업그레이드 암치료/ 후두암


KeyboardInterrupt: ignored